In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
pip install gmaps

You should consider upgrading via the '/Users/christellegeorges/opt/anaconda3/envs/PythonData/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Bitlis,38.5000,42.2500,overcast clouds,19.24,94,94,5.99,TR
1,1,Ambon,-3.6954,128.1814,scattered clouds,82.54,78,40,4.61,ID
2,2,Ushuaia,-54.8000,-68.3000,broken clouds,49.66,66,75,11.50,AR
3,3,Chokurdakh,70.6333,147.9167,few clouds,-29.24,100,15,1.79,RU
4,4,Torbay,47.6666,-52.7314,scattered clouds,14.76,80,27,4.00,CA


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
best_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
best_cities_df.head(10)

,City_ID,City,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
1,1,Ambon,-3.6954,128.1814,scattered clouds,82.54,78,40,4.61,ID
5,5,Atuona,-9.8000,-139.0333,light rain,77.43,78,21,15.01,PF
7,7,Champerico,14.3000,-91.9167,clear sky,81.36,69,5,5.66,GT
9,9,Butaritari,3.0707,172.7902,clear sky,82.06,74,2,23.15,KI
11,11,Tual,-5.6667,132.7500,broken clouds,82.13,71,65,3.96,ID
12,12,Qui Nhon,13.7667,109.2333,broken clouds,78.62,78,67,3.89,VN
16,16,Labuhan,-6.8844,112.2051,moderate rain,79.83,86,100,5.68,ID
17,17,Acu,-5.5767,-36.9086,overcast clouds,77.09,78,99,5.73,BR
20,20,Bubaque,11.2833,-15.8333,clear sky,77.92,65,9,4.92,GW
21,21,Saint-Philippe,-21.3585,55.7679,broken clouds,80.19,74,67,10.78,RE


In [21]:
import numpy as np

In [22]:
# 4a. Determine if there are any empty rows.
best_cities_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Description    0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nan_value = "NaN"
best_cities_df.replace("", nan_value, inplace=True)
best_cities_df.dropna(subset = ["Country"], inplace=True)
best_cities_df.count()

/Users/christellegeorges/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/Users/christellegeorges/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID        208
City           208
Lat            208
Lng            208
Description    208
Max Temp       208
Humidity       208
Cloudiness     208
Wind Speed     208
Country        208
dtype: int64

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = best_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ambon,ID,82.54,-3.6954,128.1814,
5,Atuona,PF,77.43,-9.8000,-139.0333,
7,Champerico,GT,81.36,14.3000,-91.9167,
9,Butaritari,KI,82.06,3.0707,172.7902,
11,Tual,ID,82.13,-5.6667,132.7500,
12,Qui Nhon,VN,78.62,13.7667,109.2333,
16,Labuhan,ID,79.83,-6.8844,112.2051,
17,Acu,BR,77.09,-5.5767,-36.9086,
20,Bubaque,GW,77.92,11.2833,-15.8333,
21,Saint-Philippe,RE,80.19,-21.3585,55.7679,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ambon,ID,82.54,-3.6954,128.1814,NaN
5,Atuona,PF,77.43,-9.8000,-139.0333,NaN
7,Champerico,GT,81.36,14.3000,-91.9167,NaN
9,Butaritari,KI,82.06,3.0707,172.7902,NaN
11,Tual,ID,82.13,-5.6667,132.7500,NaN
12,Qui Nhon,VN,78.62,13.7667,109.2333,NaN
16,Labuhan,ID,79.83,-6.8844,112.2051,NaN
17,Acu,BR,77.09,-5.5767,-36.9086,NaN
20,Bubaque,GW,77.92,11.2833,-15.8333,NaN
21,Saint-Philippe,RE,80.19,-21.3585,55.7679,NaN


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))